<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Collapsed_Emotion_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project
#check if drive is loaded successfully
!ls Data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project
'Data Exploration.ipynb'
 data_fixed_train_complement_analysis.csv
 data_fixed_train.json
 data_sample_100.json
 data_sample_10_complement_analysis.csv
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_fina

In [ ]:
# install supporting libraries
!pip install transformers

     |████████████████████████████████| 757kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 24.5MB/s 
     |████████████████████████████████| 1.1MB 45.9MB/s 
     |████████████████████████████████| 890kB 57.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b409e01a48cdbb0f88bf0a18463bd165a3fe0555686b51c571ac763b9c19f198
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
dataset_name = "model2"
model_name = "model2"
model_type = "collapsed_emotion" #pos_neg or emotion or valence or collapsed_emotion

In [ ]:
import pandas as pd
import numpy as np
train_filename = "Data/data_sample_fixed_processed_" + dataset_name + "_final.csv"
valid_filename = "Data/valid_fixed_processed_" + dataset_name + "_final.csv"
test_filename = "Data/test_fixed_processed_" + dataset_name + "_final.csv"

train_df = pd.read_csv(train_filename, sep='|', error_bad_lines=False)
train_df = train_df.dropna()
valid_df = pd.read_csv(valid_filename, sep='|', error_bad_lines=False)
valid_df = valid_df.dropna()
test_df = pd.read_csv(test_filename, sep='|', error_bad_lines=False)
test_df = test_df.dropna()

In [ ]:
category2index = {
    "sentimental":1,
    "sad":2,
    "devastated":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "afraid":6,
    "proud":7,
    "grateful":8,
    "lonely":9,
    "surprised":10,
    "anxious":11,
    "guilty":12,
    "trusting":13,
    "nostalgic":14,
    "excited":15,
    "disgusted":16,
    "caring":17,
    "furious":18,
    "angry":18,
    "prepared":19,
    "disappointed":20,
    "faithful":21,
    "confident":22,
    "apprehensive":23,
    "jealous":24,
    "embarrassed":25,
    "annoyed":26,
    "ashamed":27,
    "content":28,
    "anticipating":29
}
index2category = {}
for key, value in category2index.items():
  index2category[value] = key

def category2label(category):
  for key in category2index.keys():
    if key == category:
      return category2index[key]
  print(category)
  return None

def label2emotion(labels):
  valence_list = []
  for label in labels:
    valence_list.append(index2category[label])
  return valence_list


In [ ]:
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
print('Number of valid sentences: {:,}\n'.format(valid_df.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

train_df['label'] = train_df['emotion'].apply(lambda x: int(category2label(x.lower())))
valid_df['label'] = valid_df['emotion'].apply(lambda x: int(category2label(x.lower())))
test_df['label'] = test_df['emotion'].apply(lambda x: int(category2label(x.lower())))

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 19,533

Number of valid sentences: 2,758

Number of test sentences: 2,538



,emotion,processed_emotion,text,vader_neg,vader_neu,vader_pos,vader_compound,textblob,flair_value,flair_confidence,label
5800,impressed,positive,My tattoo artist really made my new tattoo pop...,0.000,0.804,0.196,0.7650,0.307273,POSITIVE,0.975753,5
4539,embarrassed,negative,I passed gas while on my first date with a gir...,0.070,0.930,0.000,-0.3400,0.037500,NEGATIVE,0.900509,25
8049,proud,positive,I was eating lunch with family the other day a...,0.000,0.810,0.190,0.8020,0.183333,NEGATIVE,0.989474,7
17019,surprised,positive,I was happily shocked my aunt got me a laptop....,0.135,0.674,0.191,0.4404,-0.233333,POSITIVE,0.997043,10
18143,lonely,negative,I moved an hour away from my sister. We lived ...,0.000,1.000,0.000,0.0000,-0.008333,POSITIVE,0.999934,9
1487,sad,negative,Today is my last day working. I am going to mi...,0.127,0.873,0.000,-0.2960,0.000000,POSITIVE,0.645867,2
6115,confident,positive,I think the Eagles can win the superbowl again...,0.000,0.650,0.350,0.9323,0.750000,POSITIVE,0.794800,22
5537,hopeful,positive,i know ill make it this time i failed ajob tes...,0.444,0.556,0.000,-0.8316,-0.375000,NEGATIVE,0.999944,3
1469,confident,positive,I am confident that I will find a job soon I h...,0.000,0.775,0.225,0.4939,0.500000,POSITIVE,0.571378,22
18253,embarrassed,negative,Once in elementary school my skirt ripped in f...,0.118,0.745,0.137,0.1950,-0.150000,POSITIVE,0.873531,25


In [ ]:
sentences_train = train_df.text.values
labels_train = train_df.label.values
sentences_valid = valid_df.text.values
labels_valid = valid_df.label.values
sentences_test = test_df.text.values
labels_test = test_df.label.values

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
# Print the original sentence.
print(' Original: ', sentences_train[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences_train[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_train[0])))

 Original:  I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world. I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Tokenized:  ['i', 'remember', 'going', 'to', 'the', 'fireworks', 'with', 'my', 'best', 'friend', '.', 'there', 'was', 'a', 'lot', 'of', 'people', '_', 'com', '##ma', '_', 'but', 'it', 'only', 'felt', 'like', 'us', 'in', 'the', 'world', '.', 'i', 'remember', 'going', 'to', 'see', 'the', 'fireworks', 'with', 'my', 'best', 'friend', '.', 'it', 'was', 'the', 'first', 'time', 'we', 'ever', 'spent', 'time', 'alone', 'together', '.', 'although', 'there', 'was', 'a', 'lot', 'of', 'people', '_', 'com', '##ma', '_', 'we', 'felt', 'like', 'the', 'only', 'people', 'in', 'the', 'world', '.']
Token IDs:  [1045, 3342, 2183, 2000, 1996, 16080, 2007, 2026, 2190, 2767, 

In [ ]:
longer_list = []
for index, sent in enumerate(sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
sentences_train = np.delete(sentences_train, longer_list)
labels_train = np.delete(labels_train, longer_list)

max_len = 0
# For every sentence...
for sent in sentences_train:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max train sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_valid:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max valid sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_test:
    try:
      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)
    except:
      print(sent)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max test sentence length: ', max_len)

Max train sentence length:  242
Max valid sentence length:  247
Max test sentence length:  276


In [ ]:
def tokenize(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.

      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 256,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.from_numpy(labels)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])
  return(input_ids, attention_masks, labels)